In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
image = cv2.imread("sfinge.bmp") 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

In [6]:
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Structural Element for Morphological Operations
kernel = np.ones((3, 3), np.uint8)

# Erosion (Reduces noise)
eroded = cv2.erode(binary, kernel, iterations=1)

# Expansion (Refines lines)
dilated = cv2.dilate(eroded, kernel, iterations=1)

# Clears isolated noises
opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

# Fills in gaps
closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)

# Noise Reduction - Gaussian Blur
denoised = cv2.GaussianBlur(closed, (3, 3), 0)

plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(gray, cmap='gray')
plt.title("Original Gray Image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(binary, cmap='gray')
plt.title("Binary (Otsu Thresholding)")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(denoised, cmap='gray')
plt.title("Processed Fingerprint")
plt.axis('off')

plt.show()

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'erode'
> Overload resolution failed:
>  - src is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
image = cv2.imread("sfinge.bmp", cv2.IMREAD_GRAYSCALE)  # Gri tonlamaya çevrildi
_, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


def skeletonize(image):
    skeleton = np.zeros_like(image)  # Boş bir görüntü oluştur
    temp = np.zeros_like(image)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    
    while True:
        eroded = cv2.erode(image, kernel)  # Erozyon uygula
        opened = cv2.dilate(eroded, kernel)  # Genişlet (Açma işlemi)
        temp = cv2.subtract(image, opened)  # Orijinalden çıkart
        skeleton = cv2.bitwise_or(skeleton, temp)  # İskelete ekle
        image = eroded.copy()  # Güncelle

        if cv2.countNonZero(image) == 0:  # Tüm piksel işlemleri bitince çık
            break
    
    return skeleton
